In [2]:
import requests
import pandas as pd
import os
from datetime import datetime, timedelta

os.makedirs("data", exist_ok=True)

API = "2a430097-ac9b-44e4-995e-a02afb9795b2"
SOURCE = "SN68860"
BASE_URL = "https://frost.met.no/observations/v0.jsonld"

start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 12, 31)
delta = timedelta(days=30)

# Bruk ferdig aggregerte daglige verdier fra Frost
elements = {
    "mean(air_temperature P1D)": "Temperatur (°C)",  
    "mean(air_pressure_at_sea_level P1D)": "Lufttrykk (hPa)",  
    "sum(precipitation_amount P1D)": "Nedbør (mm)"
}

force_update = input("Vil du hente nye data fra Frost API? (ja/nei): ").strip().lower()

def fetch_data():
    dataframes = {label: pd.DataFrame(columns=["Dato", label]) for label in elements.values()}

    if all(os.path.exists(os.path.join("data", f"{label}.csv")) for label in elements.values()) and force_update != "ja":
        print("Bruker eksisterende filer.")
        return

    print("Henter data...")

    current_date = start_date
    while current_date <= end_date:
        batch_start = current_date.strftime("%Y-%m-%d")
        batch_end = min(current_date + delta, end_date).strftime("%Y-%m-%d")
        batch_reference_time = f"{batch_start}/{batch_end}"

        params = {
            "sources": SOURCE,
            "elements": ",".join(elements.keys()),
            "referencetime": batch_reference_time
        }

        response = requests.get(BASE_URL, params=params, auth=(API, ""))
        
        if response.status_code == 200:
            raw_data = response.json().get("data", [])
            
            for entry in raw_data:
                dato = entry["referenceTime"].split("T")[0]
                if dato > end_date.strftime("%Y-%m-%d"):
                    continue
                for obs in entry["observations"]:
                    element_id = obs["elementId"]
                    verdi = obs["value"]
                    label = elements[element_id]
                    df = dataframes[label]

                    df = pd.concat([df, pd.DataFrame([{"Dato": dato, label: verdi}])], ignore_index=True)
                    dataframes[label] = df

        else:
            print(f"Feil ved henting av data ({batch_reference_time}): {response.status_code}")
            print(response.text)

        current_date += delta

    for label, df in dataframes.items():
        df.drop_duplicates(subset="Dato", keep="last", inplace=True)
        df.sort_values("Dato", inplace=True)
        df.to_csv(os.path.join("data", f"{label}.csv"), index=False, encoding="utf-8")
        print(f"Lagret til: data/{label}.csv")

fetch_data()


Henter data...


C:\Users\leahs\AppData\Local\Temp\ipykernel_5812\2719742325.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([{"Dato": dato, label: verdi}])], ignore_index=True)


Lagret til: data/Temperatur (°C).csv
Lagret til: data/Lufttrykk (hPa).csv
Lagret til: data/Nedbør (mm).csv


In [ ]:
# TEMPFIL 15MIN -> GJENOMSNITT DAG
#KAN EGT SLETTE!!!

import pandas as pd
# Leser inn filen med temperaturmålinger
df = pd.read_csv("../data/frost_data.csv")

# Beregn gjennomsnittstemperatur per dag
daglig_gjennomsnitt = df.groupby("Dato")["Temperatur (°C)"].mean().reset_index()

# Lagre til ny CSV-fil
daglig_gjennomsnitt.to_csv("../data/frost_air_temperature_daglig_gjennomsnitt.csv", index=False)


print("Daglig gjennomsnitt lagret i frost_air_temperature_daglig_gjennomsnitt.csv")


Daglig gjennomsnitt lagret i frost_air_temperature_daglig_gjennomsnitt.csv
